In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import tqdm
import zipfile
import torch
import pandas as pd


/home/hjmuizelaar/scriptie-310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.device_count()

0

In [3]:
from platform import python_version

print(python_version())

3.10.2


In [4]:
tokenizer = AutoTokenizer.from_pretrained("../opus-mt-nl-en")

model = AutoModelForSeq2SeqLM.from_pretrained("../opus-mt-nl-en")


In [6]:
with open('../train.sliding.full.txt') as f:
    lines = f.readlines()

In [5]:
lines_df = pd.DataFrame(lines)

In [8]:
lines_df = lines_df.rename(columns={0: 'text'})

In [9]:
lines_df.columns

Index(['text'], dtype='object')

In [10]:
from datasets import Dataset
dataset = Dataset.from_pandas(lines_df)

In [7]:
from transformers import pipeline

translator = pipeline("translation", model=model, tokenizer=tokenizer)

In [8]:
full_to_translate = []
for i, line in enumerate(lines):
    separator = "."
    new_line = line.replace("\t", separator)
    if len(new_line) > 512:
        last_index = new_line[:512].rfind(".") + 1
        if last_index < 1:
            last_index = 512
        lines_to_translate = [new_line[:last_index]]
        current_string = new_line[last_index:]
        while len(current_string) > 512:
            last_index = current_string[:512].rfind(".") + 1
            if last_index < 1:
                last_index = 512
            lines_to_translate.append(current_string[:last_index])
            current_string = current_string[last_index:]
        if len(current_string) > 0:
            lines_to_translate.append(current_string)
        full_to_translate.append(lines_to_translate)
        del(lines_to_translate)
        continue
    full_to_translate.append(new_line)

In [25]:
import os

current_to_translate = full_to_translate
if os.path.isfile('translated_output.txt'):
    with open('translated_output.txt', 'r') as f:
        translated_lines = f.readlines()
        current_index = len(translated_lines)
        if current_index < len(full_to_translate):
            current_to_translate = full_to_translate[current_index:]
print(current_index)

0


In [10]:
results = []
for i, item in enumerate(full_to_translate):
    if isinstance(item, str):
        results.append([i, 1, item])
        continue
    for j, subitem in enumerate(item):
        results.append([i, j, subitem])
    

In [11]:
result_df = pd.DataFrame(results)

In [12]:
result_df

,0,1
0,0,"Laatste tijd niet ziek geweest, geen koorts, g..."
1,1,Datum a terme: 29-03-2021. Medische indicatie:...
2,1,"Zoon werd geboren met goede start, apgar 9/10.\n"
3,2,Werkt als gastvrouw bij ipse de brugge. Intoxi...
4,3,"Leverchemie niet afwijkend. Urine pH 5.0, eiwi..."
...,...,...
2417952,1724621,Bij zwangerschapswens: Controle TSH voor conce...
2417953,1724622,"Vlot daarna een sub-optimaal CTG, bleek volled..."
2417954,1724623,Volgende dag op roltrap in het station weer de...
2417955,1724624,Anamnese: Verkort consult: Aandachtspunten uit...


In [13]:
result_df = result_df.rename(columns={0: 'id', 1:'text'})

In [15]:
from datasets import Dataset

dataset = Dataset.from_pandas(result_df)

In [16]:
def translate_dataset(examples):
    return [[t['id'], translator(t['text'])[0]['translation_text']] for t in examples]

In [26]:
import time
import multiprocessing


def translate(file, text):
    if isinstance(text, str):
        translated = translator(text)[0]['translation_text']
        file.write('"' + translated + '"' + "\n")
        return 
    translated  = "[" + "\n".join(['"' + translator(t)[0]['translation_text'] + '"' for t in text]) + "]\n" 
    file.write(translated)
    return
    


In [27]:
from joblib import parallel_backend, Parallel, delayed

with open("translated_output.txt", "w") as output:
    with parallel_backend('threading', n_jobs=-1):
        Parallel()(delayed(translate)(output, i) for i in tqdm.tqdm(full_to_translate))





  0%|          | 0/1724626 [00:00<?, ?it/s]



  0%|          | 32/1724626 [00:06<95:10:34,  5.03it/s]



  0%|          | 48/1724626 [00:36<428:37:45,  1.12it/s]



  0%|          | 64/1724626 [01:07<622:02:36,  1.30s/it]

KeyboardInterrupt: 